In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

c:\Users\Mittal\anaconda3\envs\langchain_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""   # open-AI api key


In [16]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = FAISS.load_local(
    "../vector_store/faiss_store",
    embeddings,
    allow_dangerous_deserialization=True
)

In [17]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

In [18]:
def generate_flashcards(topic="", k=6, cards=12):
    """
    topic : keyword to search inside PDF (we can give any topic from our pdf)
    k : number of (relevant) chunks to retrieve from FAISS
    cards : number of flashcards to generate
    """

    docs = vectorstore.similarity_search(topic, k=k)
    combined_text = "\n\n".join([doc.page_content for doc in docs])

    prompt = f"""
    You are an expert study assistant.
    Generate exactly {cards} flashcards in this format ONLY:

    Q: <question>
    A: <answer>

    Use the content below:

    {combined_text}

    Flashcards:
    """

    response = llm.invoke(prompt) # asking llm to convert content into Q/A flashcards
    return response.content


In [ ]:
flashcards = generate_flashcards(
    topic="Waterfall Model",
    cards=12
)

print(flashcards) # this will successfully Q/A from the relevant chunks of the topic 


Q: What is a major drawback of the classical waterfall model regarding feedback?  
A: It has no feedback path, assuming no errors are committed during any phase, and does not incorporate error correction.

Q: Why is it difficult to accommodate change requests in the classical waterfall model?  
A: The model assumes all customer requirements can be completely defined at the beginning, but these requirements often change over time.

Q: What is one advantage of the classical waterfall model?  
A: It is easy to understand and considered the basis for other software development life cycle models.

Q: How are phases processed in the classical waterfall model?  
A: Phases are processed one at a time.

Q: What does the classical waterfall model provide in terms of project milestones?  
A: It has clear and well-understood milestones.

Q: What can result from a lengthy development cycle in the waterfall model?  
A: It can lead to delays and increased costs if requirements change or new issues ar